In [ ]:
%matplotlib widget
from IPython.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))

# Dial a Ride Model Tutorial

<img src="figures/josh-connor-H3cQClU6JC0-unsplash.jpg" alt="Drawing" style="width: 640px;"/>
<font><center><a href="https://unsplash.com/photos/H3cQClU6JC0">https://unsplash.com/photos/H3cQClU6JC0</a></center></font>


# Introduction

This notebook shows two solution methods for a static dial a ride problem by a binary optimization approach.

In particular, the scenario is given by a map, along with a number of requests with pickup and dropoff locations and desired pickup times. Moreover, a fleet of cars with initial locations and start times is spread over the map. 
The solution should assign the most possible passengers - requests - with the least possible waiting times to available cars, such that the overall win as computed from individual rides is maximal.

<img src="figures/Static_ride_hailing_pre_optimization.png" alt="Drawing" style="width: 700px;"/>

# Dial a Ride Model: Problem description

In this notebook, cars will drive with constant speed. Each driven kilometer by a car has a certain cost.
At the same time, each kilometer that a car drives with a request on board has a price. Moreover, a base price per delivered request can be set.
One optimization target is the maximization of win for all cars, which is price reduced by cost.
<img src="figures/Costs.png" alt="Drawing" style="width: 300px;"/>
For example, let us assume a car drives $7$ kilometers in total and the costs per kilometer are $0.1\$$. For this car, this means a total cost of $0.7\$$. Further assume, the car carries one passenger for a distance of $5$ kilometers and the price per kilometer for carrying a passenger is $1.5\$$. Then the total price for the car is $7.5\$$.
Therefore, the win of the car is $6.8\$$.

Each request wants to be picked up as early as possible after his desired pickup time. With delayed pickup, the request is assigned a dissatisfaction which grows with waiting time according to a dissatisfaction measure, until the request is rejected after some time span.
<img src="figures/Dissatisfaction.png" alt="Drawing" style="width: 300px;"/>
In the example above, let us assume two requests want to be picked up as early as possible, but the car has to drive $2$ kilometers to the pickup location of the closest request in a big traffic jam, so that it arrives after $6$ minutes. Further assume, the total time span until rejection is $10$ minutes, and the rejection cost after this is $10$. The car could reach the second request only after $11$ minutes, and hence the request must be rejected. According to the dissatisfaction measure above, this implies a dissatisfaction of $2$ plus a rejection cost of $10$, since only one passenger could be assigned to a car, but it can still be reached in time.
The overall dissatisfaction is therefore $12$.

The problem to solve is to assign as many requests as possible to cars, such that the overall win (i.e. price reduced by cost for each car), reduced by the overall dissatisfaction is maximal.
Since both optimization targets are contradictive, they must be weighted accordingly with constants $C_{cars},C_{requests}>0$.
<img src="figures/optimization_target.png" alt="Drawing" style="width: 800px;"/>
If $C_{cars} >> C_{requests}$, maximization of win is favored over minimization of waiting time, while for $C_{cars} << C_{requests}$, lower wins are rather tolerated to reduce request waiting time.
In the example above, let us assume for simplicity that we have one car and two passengers to be carried. However, as explained above, the car can only carry one passenger, since it can not reach the second request in time. For $C_{cars}=2, C_{requests}=1$, this implies an optimization target value of
$$
2 * 6.8 - 12 = 1.6. 
$$
Let us assume, the car would have chosen the other request, which would have taken $3$ kilometers to reach plus a traveling distance of $6$ kilometers. Then the cost would have been $0.3\$$, and the price $9\$$. This implies a win of $8.7\$$.
In the scenario, assume the request could have been reached only after $8$ minutes, which implies a dissatisfaction of $6$, and that now the first request would have to be rejected at a rejection cost of $10$.
Then the overall dissatisfaction would be $16$.
The optimization target value would be
$$
2 * 8.7 - 16 = 1.4. 
$$
The optimal assignment for the example is therefore the first one, since carrying both passengers is not possible, and carrying none of them would imply an optimization value of $-20$ due to two rejections.

The solution of a problem consists of an assignment sequence of requests to cars. As mentioned, requests which could not be assigned have to be rejected.
<img src="figures/Static_ride_hailing_post_optimization.png" alt="Drawing" style="width: 700px;"/>

# Result presentation

In the implementation below, results are presented in a tabular, and visualized with an interactive graphic.

<table>
  <tr>
    <td> <img src="figures/Result_example_report.PNG" alt="Drawing" style="width: 900px;"/>
    <td> <img src="figures/Result_example_picture.PNG" alt="Drawing" style="width: 500px;"/>
  </tr>
</table>

Desired and actual pickup times are listed for all passengers as `Pickup request` and `Pickup time`. Moreover, in the column `Car`, the tabular shows the assigned cars for all requests, or if a request was rejected. Also dropoff times as computed from ride distance and driving speed are presented as `Dropoff time`. 
Moreover, the tabular contains `Price`, `Cost`, `Win` and `Dissatisfaction` for each request. 
In the `Rejection` column, a one means that a request has been rejected, while a zero means the request could be assigned. The corresponding `Rejection costs` are shown in a separate column. 
Finally, the `Contribution` from each request is computed as `Win - Dissatisfaction`.

In the visual presentation, the axis unit is kilometers. The route of each car is plotted in a different color as an alternating sequence of dashed and solid lines. A dashed line means, that the car is free and on its way to a pickup location. Conversely, a solid lines means, that the car is currently occupied with a request, which it delivers at the corresponding dropoff location.
Each request is shown at its pickup location, and annotated with a window of possible pickup times until rejection. Each car is shown at its starting location, and annotated with its win in $\$$, its transport distance in $km$, its overall driving distance in $km$, and the list of requests assigned to the car. 

# Data Format
The data for requests and car positions is extracted from a CSV file located in the `data` sub folder. A test scenario based on New York taxi data is already provided in the file `test_data_NY.csv`. A screenshot of the required data format is given below.
<img src="figures/data_format.png" alt="Drawing" style="width: 700px;"/>
The first six columns $A-F$ contain request information. In particular, the requests are numbered starting with zero as can be seen in the column `requests`. This information is not read by the code, but the column is only a helper column for the reader. The column `PU time` contains desired pickup times for each request in seconds counted from initialization of the scenario upon click of the `Call` button in the `Setup scenario` tab as described in the next section. In particular, the current time when clicking the `Call` button is considered as start time, the desired pickup time of a request is computed as start time plus the respective `PU time`.
The columns `PU x-coord` and `PU y-coord` contain x and y coordinates of all requests pickup locations on the map. These values are interpreted in the unit [$10$ meters] and converted to kilometers. For example, an x coordinate value of $261$ will be converted to $2.61$ kilometers. Similarly, the columns `DO x-coord` and `DO y-coord` contain x and y coordinates of all requests dropoff locations on the map in the unit [$10$ meters].

The columns $H-J$ contain car specific information. As for column `requests`, column `cars` is only a helper column and not read by the code. Cars are numbered starting with zero. Columns `x-coord` and `y-coord` contain x and y coordinates of the start locations for each car in the unit [$10$ meters].
The start time for each car is assumed as the time when the scenario is initialized upon click of the `Call` button in the `Setup scenario` tab as described in the next section.

Note that cars in this implementation are only able to drive on vertical and horizontal lines. In particular, cars move to locations on the concatenation of a vertical and a horizontal line. Note that this implies movements of cars on a grid of $10*10$ meter cells. The implementation is based on the typical architecture of American cities, although probably with a different scaling.

You can also test the optimization algorithms on your own data files, provided in the explained format.
Note that the asymmetric approach needs a lot more bits. The waiting time of passengers is counted in full minutes. The maximal waiting time is determined by the parameter `tolerance_interval` in the `Setup scenario` tab of the dashboard below. In particular, the maximal waiting time is twice the number chosen in `tolerance_interval`. The asymmetric approach needs $C*((R+1)*R*T + 1)$ bits, where $C$ is the number of cars, $R$ the number of cars, and $T$ the number of possible waiting times for each request. For example, a scenario with $3$ cars, $6$ requests and a `tolerance_interval` value of $4$ (i.e. $C=3,R=6$ and $T=8$) would imply $3*(7*6*8 + 1) = 1011$ bits already. The symmetric approach requires only $C*R^2$ bits, which implies $3*6^2 = 108$ bits for the same scenario. Hence, for bigger scenarios, you should consider to only execute the symmetric approach in the `Compose model` tab of the dashboard below, since annealing for QUBOs bigger than $4000$ bits takes a long time with many iterations but might not yield optimal results for less iterations. 

# Model usage
In the following code block the model is implemented and loaded in a dashboard. Classes for requests and cars are implemented as `Request` and `Car`, and allow to model single customer requests and the car properties and activities including individual tour planning. The main part is implemented in the class `Darp` inheriting form `OptimizerModel`. Provided as argument to an `Optimizer` object upon initialization, the latter creates a dashboard for testing and tuning the model.

The following tabs are included in the dashboard:
### Setup scenario
In this tab, the whole scenario for optimization is loaded upon press of the `Call` button. The data is loaded in from a CSV file, which can be selected under `Data File`. `number_cars` determines the number of cars to be extracted from the file. The extracted number of requests can be chosen proportional to the number of cars via `requests_per_car`. The constant speed that each car can drive is chosen via `speed`. Costs and prices for driven kilometers can be determined by `costs_per_km`, `price_per_km` and the base price for a delivered requests is defined by `base_price`.
The maximal dissatisfaction costs of a request is determined by `dissatisfaction_costs`. Three dissatisfaction measures are implemented and can be chosen via `tolerance_model`. The option `no_tolerance` adds the value of `dissatisfaction_costs` for each request which is not picked up in time but faster than twice the `tolerance_interval` Otherwise, the value of `rejection_costs` is added. The measure corresponding to option `half_linear` implies no dissatisfaction cost for waiting times less than `tolerance_interval`, and then increases in a linear way up to half the `dissatisfaction_costs`. For waiting times more than twice the `tolerance_interval`, the the value of `rejection_costs` is added. 
Finally, the measure for option `flat_linear` implies no dissatisfaction cost for waiting times less than `tolerance_interval`, then increases in a linear way up to the `dissatisfaction_costs`, and for waiting times more than twice the `tolerance_interval`, the value of `rejection_costs` is added. 

### Compose model
This tab is responsible to start the optimization process. The parameter `solution_approach` can be used to determine the solution approach(es) to be executed. The option `asymmetric_symmetric` invokes both, the asymmetric and the symmetric algorithm. Similarly, options `asymmetric` and `symmetric` invoke the asymmetric respectively the symmetric approach only. By clicking the `Call` button, the QUBO for the chosen approach(es) is /are build in the `Build QUBO` tab, and optimized in the `Anneal tracker` tab in the respective sub-tabs. Finally, an overview over the required times of the different steps is printed.

### Build QUBO
In the sub-tab `asymmetric`, weights for the different QUBO terms of the asymmetric approach can be set. In particular, the `weight_cost` is the parameter which is multiplied to the term which aims for minimization of the net cost, `weight_waiting` is the scalar which is multiplied to the term which aims for minimization of waiting times and rejections, and `weight_penalties` is the value which is multiplied to all terms which are responsible for achieving feasible solutions.
`annealer_version` is the Digital Annealer version which is assumed in the annealing step, and `bit_precision` is the bit precision in which the QUBO is converted before annealing.

In the sub-tab `symmetric`, weights for the different QUBO terms of the symmetric approach as described above can be set.

### Solve annealing
In the respective sub-tabs, annealing parameters for the two approaches can be defined. If a solution is not optimal after pressing the `Call` button in the `Compose model` tab, QUBO parameters can be changed in the respective `Build QUBO` sub-tab and the `Call` button can be pressed to invoke generation of a new QUBO. Then, annealing parameters can be adapted in the respective `Solve annealing` sub-tab. Pressing the `Call` button, invokes a new annealing process for the respective approach. 

### Anneal track
In this tab, a detailed overview for times of the different optimization approaches can be seen.

### Report Asymmetric
This tab shows the result which was achieved by the asymmetric optimization approach. It should be compared to the results in the symmetric approach.

### Report Symmetric
This tab shows the result which was achieved by the symmetric optimization approach. It should be compared to the results in the asymmetric approach.

### Visualization Asymmetric
This tab visualizes the routes of all assigned cars from the result of the asymmetric optimization approach. This should be compared to the visualization of the symmetric approach. Each request is plotted with its desired pickup time and the latest possible pickup time before rejection. Each car is annotated at its starting location along with its win, the distance which it traveled with requests on board, the overall distance it traveled, and the sequence of requests it serviced. The plot is interactive, meaning that you can zoom into subsections of the figure. To do this, click on the white square in the bottom left of the tab. A black cross is shown upon moving the mouse above the figure. Holding the left mouse button clicked, you can select the desired area in which you want to zoom. The zoom is applied by releasing the left mouse button. To obtain the original plot, click on the house in the lower left of the tab.
The figure can also be resized. To to this, click on the triangle in the lower right of the tab, and drag the the figure to the desired size. Again, the original plot is obtained by clicking the house in the lower left of the tab.

### Visualization Symmetric
This tab visualizes the routes of all assigned cars from the result of the symmetric optimization approach. This should be compared to the visualization of the asymmetric approach.
As for the asymmetric approach, the plot is interactive.

### Remark
The default parameters upon execution of the cell below are already tuned for the subproblem of the file `test_data_NY.csv` consisting of the first $8$ requests and the first $4$ cars. Different parameters in the `Setup scenario` cell might require different settings in the `Build QUBO` and `Solve annealing` tabs. To solve your own scenario optimal, you might first need to learn from the respective exercises in the rest of this tutorial. With the default settings, annealing with CPU of the symmetric approach can take up to 16 seconds, while annealing of the asymmetric approach can take up to five minutes due to the increased QUBO size.

A replay for an example scenario is started upon execution of the following cell.

In [ ]:
from sources.Darp import *

scenario1 = Optimizer(Darp(persistent_file = "X_03_Dial_a_ride_example.dao"), auto_load=True, read_only=True)

The following cell initializes a dashboard in which you can experiment with your own scenarios. Initially, all settings are as in the scenario above. Your changes are stored, such that you can resume with your last executed scenario in the next session.

In [ ]:
scenario2 = Optimizer(Darp(persistent_file = "X_03_Dial_a_ride.dao"))

# Formulation as Quadratic Unconstrained Binary Optimization Problem (QUBO)
In order to formulate the assignment problem as a Quadratic Unconstrained Binary Optimization Problem (QUBO), we have to formalize it first.

Let $L$ be a set of locations and $Dist:L \times L \to \mathbb{R}$ a distance function. Further, let $R \in \mathbb{N}$ be the number of requests and $C \in \mathbb{N}$ the number of cars. For $r=0,...R-1$ let $p_r \in L$ and $d_r \in L$ be the pick-up and drop-off locations of the requests and for $c=0,...C-1$ let $s_c \in L$ be the starting positions of cars. 

The problem will be solved for two different QUBOs. In this description, we only describe the symmetric approach in all details, which assumes that each car can be assigned up to two requests at most.
The asymmetric approach, in which each car can be assigned an arbitrary number of requests, is also implemented. Its formulation as a QUBO is a lot more involved and is therefore omitted. The Results of both approaches on the same data set can be compared below. 
Note that the optimization result of the asymmetric approach (in theory) should be at least as good as that of the symmetric approach, since an assignment of the symmetric approach is also feasible for the asymmetric approach.

## Symmetric approach
It is assumed, that a car can fulfill zero, one or two requests in the planned horizon. 

For $c \in \{0, 1, ... C-1 \}$ and $r, s \in \{0, 1, ... R-1\}$ with $r \ne s$ let $x_{c, r,s} = 1$ iff the car with index $c$ first fulfills request $r$ and then $s$. For two identical indices $r \in \{0, 1, ... R-1\}$ set $x_{c,r,r}=1$ iff car $c$ only fulfills request $r$.
The following constraints have to be satisfied in order to obtain feasible configurations.

### Constraint 1:
Every request can be fulfilled by at most one car either in single service or double service operation:
$$
H_1 = \sum_{r=0}^{R-1}
        \left(
          \left( 
            \left( 
              \sum_{c=0}^{C-1}
                \left( 
                  \sum_{s=0}^{R-1} x_{c, r, s} +
                  \sum_{\substack {s=0 \\ s \ne r}}^{R-1} x_{c, s, r} 
                \right) 
            \right) 
            - 1 
          \right) * 
          \left( 
            \sum_{c=0}^{C-1}
              \left( 
                \sum_{s=0}^{R-1} x_{c, r, s} +
                \sum_{\substack {s=0 \\ s \ne r}}^{R-1} x_{c, s, r}  
              \right) 
          \right)
        \right) \stackrel ! = 0
$$

### Constraint 2:
Every car can do none, one single or double transportation: 

$$
H_2 = \sum_{c=0}^{C-1}
        \left(
          \left(
            \sum_{r=0}^{R-1}\sum_{s=0}^{R-1} x_{c, r, s}
            - 1
          \right) *
          \left(
            \sum_{r=0}^{R-1}\sum_{s=0}^{R-1} x_{c, r, s}
          \right) 
        \right)
        \stackrel ! = 0
$$


The target function to be maximized is the following.
### Maximize:
Let $w_{c,r,r}$ be the win (possibly negative), when car $c$ fulfills exactly request $r$ and $w_{c,r,s}$ the win (possibly negative), when car $c$ fulfills first request $r$ and then request $s$. Note that each win is computed as the price minus the cost for the driven distance of the car $c$, weighted by $C_{cars}$.
In the implementation below, $C_{cars}=1$.
similarly, let $d_{c,r,r}$ be the dissatisfaction of request $r$, when car $c$ fulfills exactly request $r$, and $d_{c,r,s}$ the dissatisfaction, when car $c$ fulfills first request $r$ and then request $s$. Note that each dissatisfaction is computed as the sum of dissatisfactions of each passenger in case the car can reach every request in time, or the corresponding rejection cost for requests which can not be reached in time. The values $d_{c,r,r}, d_{c,r,s}$ are already multiplied by $C_{requests}$, but in the implementation below, $C_{requests}=1$.
The total target function to be maximized is:
$$
H_3 = \sum_{c=0}^{C-1}
        \sum_{r=0}^{R-1}
          \sum_{s=0}^{r-1} (w_{c, r, s} - d_{c, r, s}) x_{c, r, s} 
$$

### QUBO:
Let $A$, $B$ and $C$ be positive real numbers with A, B > C (better estimation of against maximum possible win t.b.d.) then the QUBO is

$$
H = A H_1 + B H_2 - C H_3
$$

# Code

Since the code for the implementation above is lengthy, it is divided in separate Python files.

You can find the code for the helper classes `Car` and `Request` by clicking

[Car, Request](./sources/cars_requests.py)

The symmetric model is implemented under

[Symmetric model](./sources/SymmetricModel.py)

The code for the asymmetric model is shown upon clicking

[Aymmetric model](./sources/AsymmetricModel.py)

Finally, the code of the class `Darp`, which integrates both solution approaches in a single `OptimizerModel` is implemented under

[Darp](./sources/Darp.py)

To open the file tree to all above Python files in order to open them in edit mode, please click

[Sources](./sources)